In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
from astromodal.config import load_config
from tqdm import tqdm
import polars as pl
import random
from pathlib import Path

In [6]:
config = load_config("config.yaml")

print(config['datacubes_paths'])
config

/home/astrodados4/downloads/hypercube/datacube_*.parquet


{'datacubes_paths': '/home/astrodados4/downloads/hypercube/datacube_*.parquet',
 'models_folder': '/home/schwarz/projetoFM/models',
 'hdd_folder': '/home/astrodados4/downloads/projetin'}

In [7]:
hdd_folder = config['hdd_folder']

hddfolder = Path(config["hdd_folder"]) / "image_latents" 

In [8]:
all_latents = None
for key, file in enumerate(hddfolder.glob("*.parquet")):
    latents = pl.read_parquet(file, use_pyarrow=True)
    
    if all_latents is None:
        all_latents = latents
    else:
        all_latents = pl.concat([all_latents, latents])
        
    if key == 100:
        break

In [9]:
from astromodal.tokenizers.resvq import ResidualVQ

In [10]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm 
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"

latents_tensor = torch.tensor(all_latents['latent'].to_numpy(), device=device)

D = latents_tensor.shape[1]
latents = F.normalize(latents_tensor, dim=1)


In [11]:
print("mean ||z||:", latents.norm(dim=1).mean().item())
print("std  ||z||:", latents.norm(dim=1).std().item())
print("mean z:", latents.mean().item(), "std z:", latents.std().item())

mean ||z||: 1.0
std  ||z||: 5.756724519301315e-08
mean z: -0.02639772742986679 std z: 0.01308493409305811


In [23]:
import torch

loader = DataLoader(TensorDataset(latents), batch_size=2048, shuffle=True, drop_last=True)

rvq = ResidualVQ(dim=D, num_stages=6, codebook_size=512, decay=0.99).to(device)
    
rvq.train()

def recon_errors(z, z_hat):
    # mean squared L2 error: E ||z - z_hat||^2
    mean_l2_sq = ((z - z_hat) ** 2).sum(dim=1).mean()
    # per-dim MSE: mean over all elements
    mse = ((z - z_hat) ** 2).mean()
    return mean_l2_sq.item(), mse.item()

for epoch in range(10):
    # Running sums for epoch metrics
    n_sum = 0
    mse_sum = 0.0
    l2_sum = 0.0

    # Correct dead-code tracking across the whole epoch
    counts = torch.zeros(rvq.R, rvq.K, device=device)

    for (x,) in loader:
        x = x.to(device)

        out = rvq(x, update_ema=True)
        z_q = out["z_q"]
        codes = out["codes"]   # [B, R]

        # recon metrics (batch)
        mean_l2_sq, mse = recon_errors(x, z_q)

        bs = x.size(0)
        n_sum += bs
        mse_sum += mse * bs
        l2_sum += mean_l2_sq * bs

        # accumulate token usage counts per stage
        for s in range(rvq.R):
            counts[s].scatter_add_(
                0,
                codes[:, s],
                torch.ones_like(codes[:, s], dtype=counts.dtype)
            )

    epoch_mse = mse_sum / n_sum           # per-dim MSE
    epoch_l2 = l2_sum / n_sum             # mean squared L2

    dead = (counts == 0).sum(dim=1).tolist()

    with torch.no_grad():
        dead0 = (counts[0] == 0).nonzero(as_tuple=False).squeeze(1)
        if dead0.numel() > 0:
            # reseed dead codes from random data points
            ridx = torch.randint(0, latents.shape[0], (dead0.numel(),), device=device)
            rvq.embed[0, dead0].copy_(latents[ridx].to(device))
    
    with torch.no_grad():
        e = rvq.embed[0]              # [K, D]
        rvq.embed[0].copy_(e / (e.norm(dim=1, keepdim=True) + 1e-8))
        
    print(
        f"Epoch {epoch:02d} | mean_sq_L2={epoch_l2:.6f} | per_dim_MSE={epoch_mse:.8f} "
        f"| dead_codes_per_stage={dead}"
    )
    
    # with torch.no_grad():
    #     c0 = counts[0]
    #     used0 = (c0 > 0).sum().item()
    #     p = c0 / c0.sum()
    #     p = p[p > 0]
    #     entropy_bits = (-(p * p.log()).sum() / torch.log(torch.tensor(2.0, device=device))).item()
    #     eff = 2 ** entropy_bits
    #     print(f"stage0 used={used0}/512 | entropy={entropy_bits:.2f} bits | eff_codes~{eff:.1f}")
    for s in range(rvq.R):
        cs = counts[s]
        p = cs / cs.sum()
        p = p[p>0]
        ent_bits = (-(p*p.log()).sum() / torch.log(torch.tensor(2.0, device=device))).item()
        eff = 2**ent_bits
        used = (cs>0).sum().item()
        print(s, used, ent_bits, eff)

Epoch 00 | mean_sq_L2=0.114141 | per_dim_MSE=0.00009908 | dead_codes_per_stage=[451, 0, 0, 0, 0, 0]
0 61 3.2726593017578125 9.664260223656525
1 512 6.732638359069824 106.34720884607334
2 512 6.44869327545166 87.3474254486766
3 512 6.518250465393066 91.66191142330456
4 512 6.487922191619873 89.75511117874211
5 512 6.415622711181641 85.36795479574738
Epoch 01 | mean_sq_L2=0.101388 | per_dim_MSE=0.00008801 | dead_codes_per_stage=[441, 7, 17, 27, 34, 36]
0 71 3.390507221221924 10.486833532841914
1 505 6.947203159332275 123.40039132633322
2 495 6.4447197914123535 87.10718335114065
3 485 6.431288242340088 86.29997535764055
4 478 6.421722888946533 85.72968212402763
5 476 6.375598907470703 83.03219358363746
Epoch 02 | mean_sq_L2=0.100963 | per_dim_MSE=0.00008764 | dead_codes_per_stage=[433, 2, 1, 2, 6, 5]
0 79 3.3804008960723877 10.41362818033081
1 510 6.880362033843994 117.81358003317513
2 511 6.398797512054443 84.37814769017976
3 510 6.381698131561279 83.38396853416555
4 506 6.37943935394287

In [40]:
import torch

@torch.no_grad()
def kmeans(latents, K, iters=25, batch_size=8192):
    """
    latents: [N, D] (torch tensor, normalized if desired)
    returns:
      centroids: [K, D]
      assignments: [N]
    """
    device = latents.device
    N, D = latents.shape

    # --- init: random samples
    perm = torch.randperm(N, device=device)
    centroids = latents[perm[:K]].clone()  # [K, D]

    for _ in range(iters):
        counts = torch.zeros(K, device=device)
        new_centroids = torch.zeros_like(centroids)

        for i in range(0, N, batch_size):
            x = latents[i:i+batch_size]  # [B, D]

            # squared L2 distances
            x2 = (x ** 2).sum(dim=1, keepdim=True)
            c2 = (centroids ** 2).sum(dim=1).unsqueeze(0)
            dist = x2 + c2 - 2 * x @ centroids.t()

            labels = dist.argmin(dim=1)  # [B]

            for k in range(K):
                mask = labels == k
                if mask.any():
                    new_centroids[k] += x[mask].sum(dim=0)
                    counts[k] += mask.sum()

        # avoid empty clusters
        mask = counts > 0
        centroids[mask] = new_centroids[mask] / counts[mask].unsqueeze(1)

    # final assignment
    assignments = []
    for i in range(0, N, batch_size):
        x = latents[i:i+batch_size]
        dist = (
            (x**2).sum(1, keepdim=True)
            + (centroids**2).sum(1)
            - 2 * x @ centroids.t()
        )
        assignments.append(dist.argmin(dim=1))

    return centroids, torch.cat(assignments)

@torch.no_grad()
def kmeans_recon_mse(latents, centroids, assignments):
    recon = centroids[assignments]  # [N, D]
    mse = ((latents - recon) ** 2).mean().item()
    return mse

In [ ]:
device = "cuda"

K = 2048   # IMPORTANT: try several values
centroids, labels = kmeans(latents, K=K, iters=30)

@torch.no_grad()
def kmeans_recon_errors(latents, centroids, labels):
    recon = centroids[labels]  # [N, D]

    # mean squared L2 error: E ||z - z_hat||^2
    mean_l2_sq = ((latents - recon) ** 2).sum(dim=1).mean().item()

    # per-dimension MSE
    mse = ((latents - recon) ** 2).mean().item()

    return mean_l2_sq, mse




KMeans K=2048 | mean_sq_L2=9207.786133 | per_dim_MSE=7.99287033


In [42]:
mean_l2_sq, mse = kmeans_recon_errors(latents, centroids, labels)

print(
    f"KMeans K={K} | mean_sq_L2={mean_l2_sq:.6f} | per_dim_MSE={mse:.8f}"
)

KMeans K=2048 | mean_sq_L2=0.107094 | per_dim_MSE=0.00009296
